In [32]:
class MDP:
    def __init__(self, actlist, init, terminals, discount=0.9):
        self.init = init
        self.actlist = actlist
        self.terminals = terminals
        self.discount = discount
        self.states = set()
        self.reward = {}

    def Reward(self, state):
        return self.reward.get(state, 0)  # Handle missing states safely

    def Transition(self, state):
        print("Here we initialize the probability")

    def action(self, state):
        return [None] if state in self.terminals else self.actlist


In [33]:
class GridMDP(MDP):
    def __init__(self, grid, terminals, init=(0,0), discount=0.9):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        grid.reverse()
        
        # Define action list
        actlist = directions

        super().__init__(actlist, init, terminals, discount)  # Fixed superclass init call
        
        self.grid = grid
        self.rows = len(grid)
        self.columns = len(grid[0])
        
        for y in range(self.rows):
            for x in range(self.columns):
                if grid[y][x] is not None:
                    self.states.add((x, y))
                    self.reward[(x, y)] = grid[y][x]  # Store rewards

    def display(self, mapping):
        return list(reversed([
            [mapping.get((x, y), None) for x in range(self.columns)]
            for y in range(self.rows)
        ]))

    def go(self, state, function):
        state1 = vector_add(state, function)
        return if_(state1 in self.states, state1, state)

    def Transition(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return [
                (0.8, self.go(state, action)),
                (0.1, self.go(state, turn_left(action))),
                (0.1, self.go(state, turn_right(action)))
            ]

In [34]:
import operator

def vector_add(a, b):
    return tuple(map(operator.add, a, b))

def if_(test, result, alternative):
    return result if test else alternative

def update(x, **entries):
    if isinstance(x, dict):
        x.update(entries)
    else:
        x.__dict__.update(entries)  # Fixed typo in update function

directions = [(1,0), (0,1), (-1,0), (0,-1)]

def turn_left(direction):
    return directions[(directions.index(direction) + 1) % len(directions)]

def turn_right(direction):
    return directions[(directions.index(direction) - 1)]

In [35]:
directions.index((1,0))+1

1

In [36]:
grid = [[1,2,3,4],
        [5,6,7,8],
        [9,10,11,12]]

In [37]:
turn_right((1,0))

(0, -1)

In [38]:
grid.reverse()
print(*grid , sep="\n")

[9, 10, 11, 12]
[5, 6, 7, 8]
[1, 2, 3, 4]


In [39]:
grid[1][3]

8

In [40]:
vector_add((2,2) , (1,0))

(3, 2)

In [41]:
living_reward = 0
mdp1 = GridMDP([
    [living_reward, living_reward, living_reward, +1],
    [living_reward, None, living_reward, -1],
    [living_reward, living_reward, living_reward, living_reward]
], terminals=[(3,2), (3,1)])

# Test the initialization
print("States:", mdp1.states)
print("Rewards:", mdp1.reward)

States: {(0, 1), (1, 2), (2, 1), (0, 0), (3, 1), (2, 0), (3, 0), (0, 2), (2, 2), (1, 0), (3, 2)}
Rewards: {(0, 0): 0, (1, 0): 0, (2, 0): 0, (3, 0): 0, (0, 1): 0, (2, 1): 0, (3, 1): -1, (0, 2): 0, (1, 2): 0, (2, 2): 0, (3, 2): 1}


In [42]:
for a in mdp1.action((2,2)):
    print(a)

(1, 0)
(-1, 0)
(0, 1)
(0, -1)


In [45]:
u1 = {s: 0 for s in mdp1.states}
u= u1.copy()
print(u1)
print(u)

for a in mdp1.action((2,2)):
    for (p,s1) in mdp1.Transition((2,2) , a):
        print(u1[s1] , p)

{(0, 1): 0, (1, 2): 0, (2, 1): 0, (0, 0): 0, (3, 1): 0, (2, 0): 0, (3, 0): 0, (0, 2): 0, (2, 2): 0, (1, 0): 0, (3, 2): 0}
{(0, 1): 0, (1, 2): 0, (2, 1): 0, (0, 0): 0, (3, 1): 0, (2, 0): 0, (3, 0): 0, (0, 2): 0, (2, 2): 0, (1, 0): 0, (3, 2): 0}
0 0.8
0 0.1
0 0.1
0 0.8
0 0.1
0 0.1
0 0.8
0 0.1
0 0.1
0 0.8
0 0.1
0 0.1


In [46]:
# Bellman equation is used to calculate the optimal values
# in value iteration --> calculate upto k steps 

def value_iteration(mdp, iterations=9):
    u_over_time = []
    u1 = {s: 0 for s in mdp.states}  # Fixed dictionary initialization
    R, T, discount = mdp.Reward, mdp.Transition, mdp.discount

    for _ in range(iterations):
        u = u1.copy()
        for s in mdp.states:
            if s in mdp.terminals:
                u1[s] = R(s)
            else:
                u1[s] = max(
                    sum(p * (R(s) + discount * u[s1]) for (p, s1) in T(s, a))
                    for a in mdp.action(s)
                )  # Fixed summation and function calls

        u_over_time.append(u.copy())  # Append a copy of the utility values

    return u_over_time


In [47]:
u_over_time = value_iteration(mdp1)

In [49]:
print(mdp1.Transition((2,2),(1,0)))

[(0.8, (3, 2)), (0.1, (2, 2)), (0.1, (2, 1))]


In [51]:
print(u_over_time[2])

{(0, 1): 0.0, (1, 2): 0.0, (2, 1): 0.0, (0, 0): 0.0, (3, 1): -1, (2, 0): 0.0, (3, 0): 0.0, (0, 2): 0.0, (2, 2): 0.7200000000000001, (1, 0): 0.0, (3, 2): 1}


In [57]:
print(*mdp1.display(u_over_time[4]) , sep="\n")

[0.3732480000000001, 0.6583680000000002, 0.8291880000000001, 1]
[0.0, None, 0.5136120000000002, -1]
[0.0, 0.0, 0.30844800000000006, 0.0]


In [58]:
for y in range(mdp1.columns):
    for x in range(mdp1.rows):
        print(u_over_time[1].get((x,y),None))

0.0
0.0
0.0
0.0
None
0.0
0.0
0.0
0.0
None
None
None
